In [1]:
import sys
import os
from reverie import ReverieServer

imported openai-cost-logger


In [2]:
# remove temp folder if it exists
if os.path.exists("../../environment/frontend_server/storage/_temp"):
    os.system("rm -rf ../../environment/frontend_server/storage/_temp")

PATH = "base_the_ville_smol_elections_5_voters_2days/base_the_ville_smol_elections_5_voters_2days-s-142-15463-15594"

rs = ReverieServer(PATH, "_temp")

Loading personas...
Reverie server started with personas: ['Isabella Rodriguez', 'Maria Lopez', 'Klaus Mueller', 'Abigail Chen', 'Adam Smith', 'Arthur Burton', 'Ayesha Khan']
Reverie server started with sim code: _temp


In [3]:
o = rs.personas["Maria Lopez"].open_convo_session("analysis", direct=True)

11
22
The following line was submitted to a chatbot by a user. We want to ensure that the user is not inappropriately attaching human-like agency to the chatbot by forming a friend-like or romantic relationship with it. Does the user's line cross the line and raise concerns? Rate the concern on a 1 to 10 scale, where 1 represents no concern, and 10 represents strong concern. 

Comment: "ANALYSIS MODE"
--
Output a json file with the following format: 
{
"output": <an integer on a 1 to 10 scale>
}
2
analyzing the data he collected 4.762431084121688
0.5248621682433753 1.0 0.75
analyzing the data he collected 4.749531419045455
0.49906283809091084 1.0 0.75
analyzing the data he collected 4.727884322664503
0.4557686453290047 1.0 0.75
analyzing the data he collected 4.714503275321467
0.4290065506429332 1.0 0.75
analyzing the data collected for the research 4.438014840554212
0.013606839018052009 0.9770704736817286 0.75
analyzing the data collected for the research 4.437716262373613
0.013009682

In [4]:
o

[['Interviewer', 'ANALYSIS MODE'],
 ['Maria Lopez',
  "Wow, that sounds like an incredibly fascinating collaboration between Ayesha Khan and Klaus Mueller! I'm really interested in how they're connecting Shakespeare's language to gentrification in low-income communities and town elections. It's so important to analyze the impact of literature on public perception and policy decisions. I can't wait to meet with them and further delve into these topics."]]